# Sync REDCap To CommCare 

A playground for testing `sync_redcap_to_commcare.py` and its utilities in `redcap_sync.py`

In [3]:
import os

import redcap

from cc_utilities.command_line.sync_redcap_to_commcare import get_redcap_state
from cc_utilities.redcap_sync import (
    collapse_checkbox_columns,
    normalize_phone_cols,
    set_external_id_column,
    upload_complete_records,
    upload_incomplete_records,
    split_complete_and_incomplete_records,
    add_reject_status_columns,
    redcap_import,
)

In [4]:
redcap_api_url = os.getenv("REDCAP_API_URL")
redcap_api_key = os.getenv("REDCAP_API_KEY")
commcare_api_key = os.getenv("COMMCARE_API_KEY")
commcare_user_name = os.getenv("COMMCARE_USERNAME")
commcare_project_name = os.getenv("COMMCARE_PROJECT")
database_url = os.getenv("DB_URL")

state_file = "redcap_test.yaml"
sync_all = True
phone_cols = []
external_id_col = "cdms_id"

In [5]:
# Get REDCap records

state = get_redcap_state(state_file)
redcap_project = redcap.Project(redcap_api_url, redcap_api_key)
redcap_records = redcap_project.export_records(
    # Tell PyCap to return a pandas DataFrame.
    format="df",
    df_kwargs={
        # Without index_col=False, read_csv() will use the first column
        # ("record_id") as the index, which is problematic because it's
        # not unique and is easier to handle as a separate column anyways.
        "index_col": False,
        # We import everything as a string, to avoid pandas coercing ints
        # to floats and adding unnecessary decimal points in the data when
        # uploaded to CommCare.
        "dtype": str,
    },
)

redcap_records

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,tested_positive_last_year,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test
0,1,123456789,NaN,MatchingIDAndDOB,matchymatch,en,NaN,(845)599-5999,test@test.com,2007-10-11,...,no,NaN,NaN,NaN,12345670,Testing,(999) 999-1091,2020-10-27,NaN,NaN
1,2,987654321,NaN,TestRecord2,NaN,NaN,NaN,NaN,NaN,1964-05-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,223456789,NaN,VinodTestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN
3,4,223456789,NaN,Vinod2TestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN
4,5,100500819,NaN,Test,NaN,NaN,NaN,NaN,NaN,1953-03-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,100300338,NaN,Derrick,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,100300340,NaN,Rodney,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,100501819,NaN,Test,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Match Records in CDMS

For testing what's inside `match_records_in_cdms()`

In [6]:
# Imports
from sqlalchemy import MetaData, Table, and_, create_engine, select, or_
from cc_utilities.constants import DOB_FIELD
from pprint import pprint

In [7]:
# Test data
external_id = redcap_records["cdms_id"][0]
dob = redcap_records["dob"][0]

external_id_col = "cdms_id"
db_url = database_url
table_name = "patient"

print(f"CDMS_ID: {external_id},\nDOB: {dob}")

CDMS_ID: 123456789,
DOB: 2007-10-11


In [8]:
# Drop rows missing DOB or External ID
df = redcap_records.dropna(subset=[external_id_col, DOB_FIELD])
df

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,tested_positive_last_year,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test
0,1,123456789,NaN,MatchingIDAndDOB,matchymatch,en,NaN,(845)599-5999,test@test.com,2007-10-11,...,no,NaN,NaN,NaN,12345670,Testing,(999) 999-1091,2020-10-27,NaN,NaN
1,2,987654321,NaN,TestRecord2,NaN,NaN,NaN,NaN,NaN,1964-05-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,100500819,NaN,Test,NaN,NaN,NaN,NaN,NaN,1953-03-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Load table
engine = create_engine(db_url)
meta = MetaData(bind=engine)
table = Table(table_name, meta, autoload=True, autoload_with=engine)


In [10]:
# Validate columns
column_names = [col.name for col in table.columns]
assert DOB_FIELD in column_names, \
    f"{DOB_FIELD} not in {table_name} table"
assert external_id_col in column_names, \
    f"{external_id_col} not in {table_name} table"


In [11]:
# Define the query
wheres = []
for record in df.itertuples():
    dob = record.dob
    external_id = getattr(record, external_id_col)
    print(f"Processing dob {dob} and id {external_id}")
    wheres.append([
        getattr(table.c, external_id_col) == external_id,
        getattr(table.c, DOB_FIELD) == dob
    ])

query = select(
    [getattr(table.c, external_id_col), 
     getattr(table.c, DOB_FIELD)]
).where(
    or_(*[and_(*where) for where in wheres])
)


Processing dob 2007-10-11 and id 123456789
Processing dob 1964-05-14 and id 987654321
Processing dob 1953-03-17 and id 100500819


In [12]:
# Execute
conn = engine.connect()
try:
    result = conn.execute(query)
    matching_records = [dict(row) for row in result.fetchall()]
finally:
    conn.close()

pprint("Got matches for: ")
pprint(matching_records)

'Got matches for: '
[{'cdms_id': '100500819', 'dob': '1953-03-17'},
 {'cdms_id': '123456789', 'dob': '2007-10-11'}]


In [13]:
# Split Records to matched/unmatched. 
matched_external_ids = [m[external_id_col] for m in matching_records]
unmatched_records = redcap_records.where(-df[external_id_col].isin(matched_external_ids)).dropna(subset=[external_id_col])
matched_records = df.where(df[external_id_col].isin(matched_external_ids)).dropna(subset=[external_id_col])

matched_records

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,tested_positive_last_year,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test
0,1,123456789,NaN,MatchingIDAndDOB,matchymatch,en,NaN,(845)599-5999,test@test.com,2007-10-11,...,no,NaN,NaN,NaN,12345670,Testing,(999) 999-1091,2020-10-27,NaN,NaN
4,5,100500819,NaN,Test,NaN,NaN,NaN,NaN,NaN,1953-03-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
unmatched_records

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,tested_positive_last_year,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test
1,2,987654321,NaN,TestRecord2,NaN,NaN,NaN,NaN,NaN,1964-05-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
reject_records = add_reject_status_columns(unmatched_records, external_id_col).dropna(axis=1)
reject_records

,record_id,cdms_id,first_name,dob,hard_to_isolate_reasons___food,hard_to_isolate_reasons___unable_to_distance,hard_to_isolate_reasons___prescription,hard_to_isolate_reasons___med_appts,hard_to_isolate_reasons___employer,hard_to_isolate_reasons___mental_health_resources,...,race___asian,race___hawaiian_pi,race___other,race___unknown,ethnicity___hispanic,ethnicity___not_hispanic,ethnicity___unknown,integration_status,integration_status_timestamp,integration_status_reason
1,2,987654321,TestRecord2,1964-05-14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,rejected_person_mismatch,2021-05-23 21:33:47,mismatched dob and cdms_id


In [21]:
reject_records.to_csv('/home/gabby/Downloads/test_records.csv')

In [23]:
# REDCap Import

redcap_project = redcap.Project(redcap_api_url, redcap_api_key)
response = redcap_project.import_records(
    to_import=reject_records,
    overwrite="normal",
    return_content="ids",
)
response


RequestException: {'error': 'The following fields were not found in the project as real data fields: integration_status, integration_status_timestamp, integration_status_reason'}

## Data transformations

In [29]:
cases_df = normalize_phone_cols(redcap_records, phone_cols)
cases_df.head()

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,tested_positive_last_year,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test
0,1,123456789,NaN,HannahTestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN
1,2,987654321,NaN,TestRecord2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,223456789,NaN,VinodTestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN
3,4,223456789,NaN,Vinod2TestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN
4,5,100500819,NaN,Test,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
cases_df = set_external_id_column(cases_df, external_id_col)
cases_df

,record_id,cdms_id,specimen_collection_date,first_name,last_name,primary_language,primary_language_other,phone_home,commcare_email_address,dob,...,integration_process,integration_reject,pcc_resolution_status,cdms_id_test,first_name_test,phone_test,onset,exposure_test,exposure_type_test,external_id
0,1,123456789,NaN,HannahTestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,123456789
1,2,987654321,NaN,TestRecord2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,987654321
2,3,223456789,NaN,VinodTestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN,223456789
3,4,223456789,NaN,Vinod2TestingRecord,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-28,NaN,NaN,223456789
4,5,100500819,NaN,Test,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100500819
5,6,100300338,NaN,Derrick,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100300338
6,7,100300340,NaN,Rodney,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100300340
7,8,100501819,NaN,Test,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100501819


In [39]:
# From split_complete_and_incomplete_records
# Drop columns where all values are missing.
cases_df.dropna(axis=1, how="all")

,record_id,cdms_id,first_name,hard_to_isolate_reasons___food,hard_to_isolate_reasons___unable_to_distance,hard_to_isolate_reasons___prescription,hard_to_isolate_reasons___med_appts,hard_to_isolate_reasons___employer,hard_to_isolate_reasons___mental_health_resources,hard_to_isolate_reasons___nothing,...,race___amer_indian_alaskan,race___asian,race___hawaiian_pi,race___other,race___unknown,ethnicity___hispanic,ethnicity___not_hispanic,ethnicity___unknown,onset,external_id
0,1,123456789,HannahTestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-27,123456789
1,2,987654321,TestRecord2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,987654321
2,3,223456789,VinodTestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-28,223456789
3,4,223456789,Vinod2TestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-28,223456789
4,5,100500819,Test,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100500819
5,6,100300338,Derrick,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100300338
6,7,100300340,Rodney,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100300340
7,8,100501819,Test,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100501819


In [40]:
complete_records, incomplete_records = split_complete_and_incomplete_records(cases_df)
complete_records

,record_id,cdms_id,first_name,hard_to_isolate_reasons___food,hard_to_isolate_reasons___unable_to_distance,hard_to_isolate_reasons___prescription,hard_to_isolate_reasons___med_appts,hard_to_isolate_reasons___employer,hard_to_isolate_reasons___mental_health_resources,hard_to_isolate_reasons___nothing,...,race___amer_indian_alaskan,race___asian,race___hawaiian_pi,race___other,race___unknown,ethnicity___hispanic,ethnicity___not_hispanic,ethnicity___unknown,onset,external_id
0,1,123456789,HannahTestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-27,123456789
2,3,223456789,VinodTestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-28,223456789
3,4,223456789,Vinod2TestingRecord,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2020-10-28,223456789


In [41]:
incomplete_records

,record_id,cdms_id,first_name,hard_to_isolate_reasons___food,hard_to_isolate_reasons___unable_to_distance,hard_to_isolate_reasons___prescription,hard_to_isolate_reasons___med_appts,hard_to_isolate_reasons___employer,hard_to_isolate_reasons___mental_health_resources,hard_to_isolate_reasons___nothing,...,race___amer_indian_alaskan,race___asian,race___hawaiian_pi,race___other,race___unknown,ethnicity___hispanic,ethnicity___not_hispanic,ethnicity___unknown,onset,external_id
1,2,987654321,TestRecord2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,987654321
4,5,100500819,Test,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100500819
5,6,100300338,Derrick,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100300338
6,7,100300340,Rodney,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100300340
7,8,100501819,Test,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,100501819


In [ ]:
# upload_complete_records(
#     cases_df, commcare_api_key, commcare_project_name, commcare_user_name
# )

In [42]:
# upload_incomplete_records(
#     cases_df, commcare_api_key, commcare_project_name, commcare_user_name
# )

for index, row in incomplete_records.iterrows():
    # Drops any values in this Series with missing/NA values,
    # and converts it back to a DataFrame.
    data = row.dropna().to_frame().transpose()

data

,record_id,cdms_id,first_name,hard_to_isolate_reasons___food,hard_to_isolate_reasons___unable_to_distance,hard_to_isolate_reasons___prescription,hard_to_isolate_reasons___med_appts,hard_to_isolate_reasons___employer,hard_to_isolate_reasons___mental_health_resources,hard_to_isolate_reasons___nothing,...,race___black,race___amer_indian_alaskan,race___asian,race___hawaiian_pi,race___other,race___unknown,ethnicity___hispanic,ethnicity___not_hispanic,ethnicity___unknown,external_id
7,8,100501819,Test,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100501819
